# Managing permissions

This example requires the Pro-feature to be active

In [7]:
import geoengine as ge

## Initialize Geo Engine as Admin

In [8]:
ge.initialize("http://localhost:3030/api", ("admin@localhost", "admin"))

## Create layer

In [9]:
root_collection = ge.layer_collection()
root_of_layerdb = root_collection.items[1].load()
layer_id = root_of_layerdb.add_layer(
    name="ports clone",
    description="test description",
    workflow={
        "type": "Vector",
        "operator": {
            "type": "PointInPolygonFilter",
            "params": {},
            "sources": {
                "points": {
                    "type": "OgrSource",
                    "params": {
                        "data": {
                            "type": "internal",
                            "datasetId": "a9623a5b-b6c5-404b-bc5a-313ff72e4e75"
                        },
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                },
                "polygons": {
                    "type": "OgrSource",
                    "params": {
                        "data": {
                            "type": "internal",
                            "datasetId": "b6191257-6d61-4c6b-90a4-ebfb1b23899d"
                        },
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                }
            }
        }
    },
    symbology=None,
)

root_of_layerdb.reload()
root_of_layerdb

Layer Collection
name: LayerDB
description: Root collection for LayerDB
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: 5e2fcab4-6b1c-4702-a6a1-1c6d4829d00e
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Login anonymously, the layer is not visible

In [10]:
ge.initialize("http://localhost:3030/api")
root_collection = ge.layer_collection()
root_of_layerdb = root_collection.items[1].load()

root_of_layerdb

Layer Collection
name: LayerDB
description: Root collection for LayerDB
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Share layer with all anonymous users

In [11]:
ge.initialize("http://localhost:3030/api", ("admin@localhost", "admin"))
resource = ge.ResourceId({
    "type": "Layer",
    "id": layer_id
})
ge.add_permission(ge.ANONYMOUS_USER_ROLE_ID, resource, ge.Permission.READ)


# Layer is now visible for anonymous users

In [12]:
ge.initialize("http://localhost:3030/api")
root_collection = ge.layer_collection()
root_of_layerdb = root_collection.items[1].load()

root_of_layerdb

Layer Collection
name: LayerDB
description: Root collection for LayerDB
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: 5e2fcab4-6b1c-4702-a6a1-1c6d4829d00e
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74